In [1]:
%pip install -q --upgrade keras keras-cv tensorflow tensorflow_datasets setuptools scikit-learn segmentation-models matplotlib opencv-python seaborn keras_tuner pandas
print("Successfully installed required packages.")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install kaggle
# !kaggle datasets download -d tawsifurrahman/tuberculosis-tb-chest-xray-dataset
!kaggle datasets download -d nikhilpandey360/chest-xray-masks-and-labels
!unzip chest-xray-masks-and-labels.zip
# !unzip tuberculosis-tb-chest-xray-dataset.zip
# !rm -rf data
!mv Lung\ Segmentation segmentation_data
!rm -rf ./segmentation_data/ClinicalReadings
!rm -rf ./data
!rm ./segmentation_data/NLM-ChinaCXRSet-ReadMe.docx
!rm ./segmentation_data/NLM-MontgomeryCXRSet-ReadMe.pdf
!mv ./segmentation_data/CXR_png ./segmentation_data/images
# !mv TB_Chest_Radiography_Database data

In [ ]:
!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y

In [2]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import segmentation_models as sm
import keras
import keras_tuner
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf

2024-08-21 09:31:33.307957: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 09:31:33.318631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 09:31:33.331212: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 09:31:33.334982: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 09:31:33.344347: I tensorflow/core/platform/cpu_feature_guar

Segmentation Models: using `tf.keras` framework.


In [ ]:
DATA_DIR = "./segmentation_data"
image_dir = os.path.join(DATA_DIR, "images")
mask_dir = os.path.join(DATA_DIR, "masks")

masks = [f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))]
for mask in masks:
    if mask.endswith("_mask.png"):
        os.rename(os.path.join(mask_dir, mask), os.path.join(mask_dir, mask.replace("_mask", "")))

images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
for img in images:
    if os.path.isfile(os.path.join(mask_dir, img)):
        continue
    os.remove(os.path.join(image_dir, img))

# Split the dataset into training and validation sets
images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
masks = [f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))]

from sklearn.model_selection import train_test_split
import shutil

images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
masks = [f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))]

train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.1)

if os.path.exists(os.path.join(DATA_DIR, "train")):
    print("Removing existing train dataset...")
    shutil.rmtree(os.path.join(DATA_DIR, "train"))

if os.path.exists(os.path.join(DATA_DIR, "val")):
    print("Removing existing validation dataset...")
    shutil.rmtree(os.path.join(DATA_DIR, "val"))

train_image_dir = os.path.join(DATA_DIR, "train", "images")
val_image_dir = os.path.join(DATA_DIR, "val", "images")
train_mask_dir = os.path.join(DATA_DIR, "train", "masks")
val_mask_dir = os.path.join(DATA_DIR, "val", "masks")

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_mask_dir, exist_ok=True)
os.makedirs(val_mask_dir, exist_ok=True)

def copy_files(file_list, src_dir, dst_dir):
    for file in file_list:
        shutil.copy(os.path.join(src_dir, file), os.path.join(dst_dir, file))

print("Copying train images...")
copy_files(train_images, image_dir, train_image_dir)
print("Copying validation images...")
copy_files(val_images, image_dir, val_image_dir)
print("Copying train masks...")
copy_files(train_masks, mask_dir, train_mask_dir)
print("Copying validation masks...")
copy_files(val_masks, mask_dir, val_mask_dir)

In [9]:
# https://www.tensorflow.org/tutorials/images/segmentation
class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=42):
        super().__init__()

        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = keras.Sequential([
            keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=seed),
            keras.layers.RandomBrightness(factor=0.2, seed=seed),
            keras.layers.RandomRotation(factor=0.2, seed=seed)
        ])
        self.augment_labels = keras.Sequential([
            keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=seed),
            keras.layers.RandomRotation(factor=0.2, seed=seed)
        ])

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)

        labels = labels / 255
        return inputs, labels
  
def augment_validation(inputs, labels):
    return inputs, labels / 255

def get_data(batch_size):
    train_images, validation_images = keras.utils.image_dataset_from_directory(
        "segmentation_data/train/images",
        labels=None,
        batch_size=batch_size,
        image_size=(256, 256),
        shuffle=False,
        validation_split=0.1,
        subset='both'
    )

    train_masks, validation_masks = keras.utils.image_dataset_from_directory(
        "segmentation_data/train/masks",
        labels=None,
        batch_size=batch_size,
        image_size=(256, 256),
        color_mode='grayscale',
        shuffle=False,
        validation_split=0.1,
        subset='both'
    )

    training_dataset = tf.data.Dataset.zip(train_images, train_masks)
    validation_dataset = tf.data.Dataset.zip(validation_images, validation_masks)

    training_dataset = training_dataset.map(Augment()).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(augment_validation).prefetch(buffer_size=tf.data.AUTOTUNE)

    return training_dataset, validation_dataset

## Resnet34

In [10]:
BACKBONE = 'resnet34'
def build_resnet_model(hp):
    loss_idx = hp.Choice('loss_idx', values=[0, 1])
    losses = [keras.losses.binary_crossentropy, sm.losses.dice_loss]
    loss = losses[loss_idx]

    optimizer_idx = hp.Choice('optimizer_idx', values=[0, 1])
    optimizers = [keras.optimizers.Adam(), keras.optimizers.SGD()]
    optimizer = optimizers[optimizer_idx]

    model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    return model

In [12]:
preprocess_input = sm.get_preprocessing(BACKBONE)

def process_dataset(images, labels):
    return preprocess_input(images), labels

for batch_size in [4, 8, 16]:
    project_name= 'resnet34-%d' % batch_size

    training_dataset, validation_dataset = get_data(batch_size)
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    tuner = keras_tuner.GridSearch(
        objective='val_loss',
        hypermodel=build_resnet_model,
        project_name=project_name
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, start_from_epoch=10)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    tuner.search(
        training_dataset,
        batch_size=batch_size,
        epochs=50,
        validation_data=validation_dataset,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

Trial 4 Complete [00h 12m 00s]
val_loss: 0.20852604508399963

Best val_loss So Far: 0.17512701451778412
Total elapsed time: 00h 32m 36s


## Inceptionv3

In [27]:
BACKBONE = 'inceptionv3'
def build_inceptionv3_model(hp):
    loss_idx = hp.Choice('loss_idx', values=[0, 1])
    losses = [keras.losses.binary_crossentropy, sm.losses.dice_loss]
    loss = losses[loss_idx]

    optimizer_idx = hp.Choice('optimizer_idx', values=[0, 1])
    optimizers = [keras.optimizers.Adam(), keras.optimizers.SGD()]
    optimizer = optimizers[optimizer_idx]

    model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    return model

In [37]:
preprocess_input = sm.get_preprocessing(BACKBONE)

def process_dataset(images, labels):
    return preprocess_input(images), labels

for batch_size in [4, 8, 16]:
    project_name = 'inceptionv3-%d' % batch_size

    training_dataset, validation_dataset = get_data(batch_size)
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    tuner = keras_tuner.GridSearch(
        objective='val_loss',
        hypermodel=build_inceptionv3_model,
        project_name=project_name
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, start_from_epoch=10)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    tuner.search(
        training_dataset,
        batch_size=batch_size,
        epochs=50,
        validation_data=validation_dataset,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

Trial 4 Complete [00h 11m 53s]
val_loss: 0.2067612260580063

Best val_loss So Far: 0.17479930818080902
Total elapsed time: 00h 33m 12s


## Original Model

In [38]:
def conv_block(inputs, num_filters):
    x = keras.layers.Conv2D(num_filters, 3, padding='same')(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)

    x = keras.layers.Conv2D(num_filters, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = keras.layers.MaxPool2D((2, 2))(x)

    return x, p

def decoder_block(inputs, skip, num_filters):
    x = keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs)
    x = keras.layers.Concatenate()([x, skip])
    x = conv_block(x, num_filters)

    return x

def build_unet(input_shape=(256, 256, 3)):
    inputs = keras.Input(shape=input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 64)
    s3, p3 = encoder_block(p2, 64)
    s4, p4 = encoder_block(p3, 64)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 512)
    d3 = decoder_block(d2, s2, 512)
    d4 = decoder_block(d3, s1, 512)

    outputs = keras.layers.Conv2D(1, 1, padding='same', activation='sigmoid')(d4)
    return keras.Model(inputs, outputs, name='U-Net')

def build_original_model(hp):
    loss_idx = hp.Choice('loss_idx', values=[0, 1, 2, 3])
    losses = [sm.losses.bce_jaccard_loss, sm.losses.jaccard_loss, sm.losses.bce_dice_loss, sm.losses.dice_loss]
    loss = losses[loss_idx]

    optimizer_idx = hp.Choice('optimizer_idx', values=[0])
    optimizers = [keras.optimizers.Adam()]
    optimizer = optimizers[optimizer_idx]

    model = build_unet()

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    return model

In [39]:
def preprocess_input(images):
    return images / 255

def process_dataset(images, labels):
    return preprocess_input(images), labels

for batch_size in [4, 8, 16]:
    project_name = 'original-%d' % batch_size

    training_dataset, validation_dataset = get_data(batch_size)
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    tuner = keras_tuner.GridSearch(
        objective='val_loss',
        hypermodel=build_original_model,
        project_name=project_name
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, start_from_epoch=10)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    tuner.search(
        training_dataset,
        batch_size=batch_size,
        epochs=50,
        validation_data=validation_dataset,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )

Trial 2 Complete [00h 19m 59s]
val_loss: 0.30447280406951904

Best val_loss So Far: 0.30447280406951904
Total elapsed time: 00h 35m 02s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
2                 |1                 |loss_idx
0                 |0                 |optimizer_idx

Epoch 1/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 76s 408ms/step - accuracy: 0.7451 - f1-score: 0.4798 - iou_score: 0.3242 - loss: 1.0716 - val_accuracy: 0.2983 - val_f1-score: 0.2961 - val_iou_score: 0.1742 - val_loss: 2.3309 - learning_rate: 0.0010
Epoch 2/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 50s 342ms/step - accuracy: 0.8385 - f1-score: 0.6208 - iou_score: 0.4523 - loss: 0.7524 - val_accuracy: 0.3074 - val_f1-score: 0.4451 - val_iou_score: 0.2869 - val_loss: 2.6043 - learning_rate: 0.0010
Epoch 3/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 50s 345ms/step - accuracy: 0.8492 - f1-score: 0.6407 - iou_score: 0.4741 - loss: 0.7089 - val_accuracy: 0.7620 - val_f1-score: 0.6625 - val_iou_score: 0.4964 - va

## EfficientNetB5

In [ ]:
BACKBONE = 'efficientnetb5'
def build_efficientnet_model(hp):
    loss_idx = hp.Choice('loss_idx', values=[0, 1])
    losses = [keras.losses.binary_crossentropy, sm.losses.dice_loss]
    loss = losses[loss_idx]

    optimizer_idx = hp.Choice('optimizer_idx', values=[0, 1])
    optimizers = [keras.optimizers.Adam(), keras.optimizers.SGD()]
    optimizer = optimizers[optimizer_idx]

    model = sm.Unet(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()]
    )

    return model

In [ ]:
preprocess_input = sm.get_preprocessing(BACKBONE)

def process_dataset(images, labels):
    return preprocess_input(images), labels

for batch_size in [4, 8, 16]:
    project_name = 'efficientnetb5-%d' % batch_size

    training_dataset, validation_dataset = get_data(batch_size)
    training_dataset = training_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.map(process_dataset).prefetch(buffer_size=tf.data.AUTOTUNE)

    tuner = keras_tuner.GridSearch(
        objective='val_loss',
        hypermodel=build_efficientnet_model,
        project_name=project_name
    )

    tensorboard_cb = keras.callbacks.TensorBoard(log_dir='tensorboard/%s' % (project_name))
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, start_from_epoch=10)
    reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(patience=5, cooldown=5)

    tuner.search(
        training_dataset,
        batch_size=batch_size,
        epochs=50,
        validation_data=validation_dataset,
        callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb]
    )